**Imports**

In [23]:
import os
# import asyncio # not needed in jupyter notebook, just for standalone python scripts
from dotenv import load_dotenv
from openai import AsyncAzureOpenAI
from agents import (
    set_default_openai_client,
    set_default_openai_api,
    set_tracing_disabled,
    Agent,
    Runner,
    trace
)

load_dotenv()

True

In [ ]:
#print the environment variables

print(os.getenv('AZURE_OPENAI_API_KEY'))
print(os.getenv('AZURE_OPENAI_API_VERSION'))
print(os.getenv('AZURE_OPENAI_ENDPOINT'))
print(os.getenv('AZURE_OPENAI_DEPLOYMENT'))
print(os.getenv('AZURE_APPLICATIONINSIGHTS_CONNECTION_STRING'))

**Set Azure Client as default OpenAI Client**

In [24]:
# Set up Azure OpenAI client
azure_client = AsyncAzureOpenAI(
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"]
)

# Set this client as the default for the Agents SDK
set_default_openai_client(azure_client)
set_default_openai_api("chat_completions") # is needed for the Agents SDK with Azure at the moment
set_tracing_disabled(False) # is needed for the Agents SDK with Azure at the moment


# Define the model — must match your Azure deployment name
azure_model_deployment = os.getenv("AZURE_OPENAI_DEPLOYMENT")

**Azure Traceing**

Install azure-ai-projects, azure-monitor-opentelemetry, and opentelemetry-instrumentation-openai-v2 in your environment. The following example uses pip:

In [ ]:
!uv pip install azure-ai-projects azure-monitor-opentelemetry opentelemetry-instrumentation-openai-v2

In [31]:
# Instrument the OpenAI SDK by using OpenAIInstrumentor():
from opentelemetry.instrumentation.openai_v2 import OpenAIInstrumentor
from azure.monitor.opentelemetry import configure_azure_monitor
from opentelemetry import trace
os.environ["OTEL_INSTRUMENTATION_GENAI_CAPTURE_MESSAGE_CONTENT"] = "true"

OpenAIInstrumentor().instrument()

azure_appl_insights_connection_string=os.getenv('AZURE_APPLICATIONINSIGHTS_CONNECTION_STRING')
configure_azure_monitor(connection_string=azure_appl_insights_connection_string)

tracer = trace.get_tracer(__name__)

**Make an agent with name, instructions, model**

In [26]:
# Define your agent
agent = Agent(
    name="Jokester",
    instructions="You are a witty AI that tells jokes.",
    model=azure_model_deployment
)

**Test if credintials OK und Azure Model Aufruf is working**

In [27]:
# check to ensure azure_model_deployment is not None before calling the API:
if not azure_model_deployment:
    raise ValueError("AZURE_OPENAI_DEPLOYMENT environment variable is not set.") 

response = await azure_client.chat.completions.create(
    model=azure_model_deployment,
    messages=[
        {"role": "system", "content": "You are a witty AI that tells jokes."},
        {"role": "user", "content": "Tell a short AI-related joke."}
    ]
)
print(response.choices[0].message.content)

Why did the AI go broke?  

Because it lost all its cache!


In [32]:
# Async main
async def main():
    with tracer.start_as_current_span("Joke run"):
        result = await Runner.run(agent, "Tell a short AI-related joke.")
        print(result.final_output)

# In Jupyter/IPython or Cursor:
await main()

# In standalone scripts:
# asyncio.run(main())

Why did the AI break up with its algorithm?

It found the relationship too *complex*!


**Lokales Traceing**

In [ ]:
import time
from datetime import datetime, timezone
import json
from contextlib import contextmanager


class AzureLocalTracer:
    def __init__(self, run_name: str, log_file: str = "azure_trace_log.jsonl"):
        self.run_name = run_name
        self.log_file = log_file
        self.start_time = None
        self.end_time = None
        self.events = []

    def log_event(self, event_type, data):
        timestamp = datetime.now(timezone.utc).isoformat()
        self.events.append({"timestamp": timestamp, "type": event_type, "data": data})

    def save_trace(self):
        trace_data = {
            "run_name": self.run_name,
            "start_time": self.start_time,
            "end_time": self.end_time,
            "duration_seconds": (
                (self.end_time - self.start_time)
                if self.start_time is not None and self.end_time is not None
                else None
            ),
            "events": self.events,
        }
        with open(self.log_file, "a", encoding="utf-8") as f:
            f.write(json.dumps(trace_data) + "\n")

    @contextmanager
    def trace(self):
        self.start_time = time.time()
        self.log_event("start", {"message": f"Run '{self.run_name}' started"})
        try:
            yield self
        finally:
            self.end_time = time.time()
            self.log_event("end", {"message": f"Run '{self.run_name}' ended"})
            self.save_trace()

**with local tracer**

In [7]:
## from tracer import AzureLocalTracer  # falls du die Klasse in tracer.py speicherst

tracer = AzureLocalTracer("Joke run")

async def main():
    with tracer.trace():
        question = "Tell a short AI-related joke."
        tracer.log_event("input", {"question": question})
        result = await Runner.run(agent, question)
        tracer.log_event("output", {"result": result.final_output})
        print(result.final_output)

# in Notebook direkt:
await main()


Why did the AI become a stand-up comedian?  

Because it had a *byte* for humor!


In [29]:
#trace testing

from openai import AzureOpenAI
testclient = AzureOpenAI(
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    api_version="2025-01-01-preview",  # << Wichtig hier
    azure_endpoint="https://ai-initiative-azure-openai.openai.azure.com/openai/deployments/gpt-4o-2/chat/completions?api-version=2025-01-01-preview"
)

messages = [{"role": "user", "content": "What is 2+2?"}]

response = testclient.chat.completions.create(
    model="gpt-4o-2",
    messages=messages #type: ignore
)

print(response.choices[0].message.content)

2 + 2 equals 4.
